In [1]:
import dask.dataframe as dd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import r2_score
import lightgbm as lgb
import dask_ml
import dask
import pandas as pd
#dask.config.set({"distributed.utils.perf.gc-fraction": 0.8})
#dask.config

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*Sending large graph.*")


In [3]:
from dask.distributed import Client, LocalCluster
import dask.multiprocessing

# Get the number of available CPU cores
n_cores = 8-1

cluster = LocalCluster(processes=True,n_workers=6, threads_per_worker=1)
client = Client(cluster)

In [4]:
folders = [
    'train0_25',
    'train25_50',
    'train50_75',
    'train75_100'
]

# Read Parquet files from each folder into Dask DataFrames
dfs = [dd.read_parquet(folder) for folder in folders]

# Concatenate all DataFrames into a single DataFrame
data = dd.concat(dfs)

In [ ]:
data.shape[0].compute()

In [ ]:
#numPartitions = data.npartitions
#splitPart = int(numPartitions*0.7)
#train = data.partitions[0:splitPart]
#test = data.partitions[splitPart:numPartitions]

In [3]:
feat60 = ['state_t', 'state_q0001','state_q0002','state_q0003','state_u','state_v','pbuf_ozone','pbuf_CH4','pbuf_N2O']

#feat60 = ['state_q0001','state_q0002','state_q0003','state_u','state_v','pbuf_ozone','pbuf_CH4','pbuf_N2O']
feat1 = ['state_ps','pbuf_SOLIN','pbuf_LHFLX','pbuf_SHFLX','pbuf_TAUX','pbuf_TAUY','pbuf_COSZRS','cam_in_ALDIF','cam_in_ALDIR','cam_in_ASDIF','cam_in_ASDIR','cam_in_LWUP','cam_in_ICEFRAC','cam_in_LANDFRAC','cam_in_OCNFRAC','cam_in_SNOWHLAND']

target60 = ['ptend_t','ptend_q0001','ptend_q0002','ptend_q0003','ptend_u','ptend_v']
target1 = ['cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

features60 = [] 
for f in feat60:
    features60 = features60 + [f+'_'+str(i) for i in range(60)]
allF = features60 + feat1

targets60 = [] 
for f in target60:
    targets60 = targets60 + [f+'_'+str(i) for i in range(60)]
allT = targets60 + target1

targetsToDrop12 = [ 'ptend_q0001', 'ptend_q0002', 'ptend_q0003', 'ptend_u', 'ptend_v']
dropT = ['ptend_q0002_12','ptend_q0002_13','ptend_q0002_14'] # attention, I think i also need to predict _15
for f in targetsToDrop12:
    dropT = dropT + [f+'_'+str(i) for i in range(12)]

allT2 = [i for i in allT if i not in dropT]

# find corrupt files

In [ ]:
import os
import dask.dataframe as dd
from dask.distributed import Client

# Optionally, start a Dask client for better error handling and performance
client = Client()

def find_parquet_files(folder):
    """Recursively find all parquet files in a given folder."""
    parquet_files = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith(".parquet"):
                parquet_files.append(os.path.join(root, file))
    return parquet_files

# Collect all Parquet files from all folders
all_parquet_files = []
all_parquet_files.extend(find_parquet_files(folders[3]))

corrupted_files = []

# Attempt to read each Parquet file individually
for file in all_parquet_files:
    try:
        df = dd.read_parquet(file)
        # Force computation to check for any reading issues
        df.head()
    except Exception as e:
        print(f"Error reading {file}: {e}")
        corrupted_files.append(file)

if corrupted_files:
    print("The following files are corrupted or not Parquet files:")
    for corrupted_file in corrupted_files:
        print(corrupted_file)
else:
    print("No corrupted files found.")


In [ ]:
len(corrupted_files)

In [ ]:
import os

for file in corrupted_files:
    os.remove(file)


# check out data

In [ ]:
import logging

In [ ]:
logging.getLogger("distributed.utils_perf").setLevel(logging.ERROR)
data['ptend_q0002_15'].compute().hist(bins=100)

# visualize data
- all variables seem to be somewhat stationary -> no time dependency in data
- we can just downsample data

In [ ]:
def createHistPlt(a, f,st):
    if not os.path.exists('histplots/'+f+st+'.jpg'):
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))   

        # Histogram
        ax1.hist(a, bins=100, edgecolor='k', alpha=0.7)
        #ax1.set_xlabel(f)
        ax1.set_ylabel('Frequency')
        ax1.set_title('Histogram of '+f)    

        # Time Series
        ax2.scatter(a.index, a, s=1, alpha=0.7)
        ax2.set_xlabel('index')
        ax2.set_ylabel(f)
        m = a.mean()
        s = a.std()
        ax2.set_title('mean'+str(round(m,2)) + ' std '+str(round(s,2)))

        # Adjust layout
        #plt.tight_layout()
        #plt.show()
        fig.savefig('histplots/'+f+st+'.jpg')
    else:
        print('skipped',f)
def createHist(a,f,st):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))   
    ax1.hist(a, bins=100, edgecolor='k', alpha=0.7)
    #ax1.set_xlabel(f)
    ax1.set_ylabel('Frequency')
    ax1.set_title('Histogram of '+f)  
    m = a.mean()
    s = a.std()
    b = (a - a.mean())/a.std()
    ax2.hist(a, bins=100, edgecolor='k', alpha=0.7)
    ax2.set_xlabel(f)
    ax2.set_ylabel('Frequency')
    ax2.set_title('mean'+str(round(m,2)) + ' std '+str(round(s,2)))
    fig.savefig('histplots/'+f+st+'.jpg')

In [ ]:
# temperature: distribution moves from left to right
for f in allF:
    if not os.path.exists('histplots/'+f+'feat'+'.jpg'):
        a = data[f].compute()
        createHistPlt(a,f, 'feat')
for f in allT2:
    if not os.path.exists('histplots/'+f+'targ'+'.jpg'):
        a = data[f].compute()
        createHistPlt(a,f, 'targ')

# downsample data

In [6]:
import numpy as np
np.random.seed(42)

orig_partitions = [i for i in range(0,int(data.npartitions))]
np.random.shuffle(orig_partitions) #shuffles inplace

trainSep = int(0.1* data.npartitions)
valEnd = int(0.015* data.npartitions) + trainSep

sampledPartIdxTrain = orig_partitions[0:trainSep]
sampledPartIdxTest  = orig_partitions[trainSep:valEnd]

In [7]:
len(sampledPartIdxTest), len(sampledPartIdxTrain)

(42, 283)

In [ ]:
import sys
size_in_bytes = sys.getsizeof(X_val)
print('in mb',size_in_bytes/1000/1000) 
#del a

# baseline approach (LGBM)
- 0.45 public score (without temp data & not shuffled between train/test)
- 0.47 public score (with temp data & shuffled properly 2*100 partitions)
- room for more improvement  (feature engineering)

In [ ]:
def convertData(partIdStart, partIdEnd, train, featuresTrain, targetFeatures, mean_values, std_values):
    X = train[featuresTrain].partitions[partIdStart:partIdEnd].compute()
    y = train[targetFeatures].partitions[partIdStart:partIdEnd].compute()
    # normalize
    for f in featuresTrain:
        X[f] = (X[f] - mean_values[f]) / std_values[f]
    for f in targetFeatures:
        y[f] = (y[f] - mean_values[f]) / std_values[f]

    return X,y

def convertDataBack(y, pred, feature, mean_values, std_values):
    cy = y*std_values[feature] + mean_values[feature]
    cpred = pred*std_values[feature] + mean_values[feature]
    return cy, cpred

def calcR2scoreFromConvData(y, pred, feature, mean_values, std_values):
    cy, cpred = convertDataBack(y, pred, feature, mean_values, std_values)
    return r2_score(cy, cpred)

In [8]:
# 8 partitions = 260mb
import pandas as pd
valList = []
for i in sampledPartIdxTest:
    valList.append(data.get_partition(int(i)).compute())
val = pd.concat(valList)
X_val = val[allF]
y_val = val[allT2]
del val, valList

In [9]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'l2',
    'num_leaves': 15,
    #'learning_rate': 0.05,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'verbose': -1
}



r2ScoreDict = {}
r2ScoreDict = {f: {} for f in allT2}

partPerLoop = 100
nsplitData = int(len(sampledPartIdxTrain)/partPerLoop)
for i in range(nsplitData):
    startPartIdx = i*partPerLoop
    dlist = []
    for j in range(partPerLoop):
        dlist.append(data.get_partition(int(sampledPartIdxTrain[startPartIdx+j])).compute())
    locdata = pd.concat(dlist)
    print('done preprocessing data')
    X = locdata[allF]

    for f in allT2:
        print('processing ',f)
        fileName = 'individualLGBMs/model_'+f+'.txt'
        gbm = lgb.Booster(model_file=fileName) if i != 0 else None

        valSet = lgb.Dataset(X_val, label=y_val[f], free_raw_data=False)
        y = locdata[f]
        train_set = lgb.Dataset(X, y, free_raw_data=False)
        gbm = lgb.train(params,
                    train_set,
                    num_boost_round=20, 
                    valid_sets=valSet,
                    init_model=gbm)
        
        predTrain = gbm.predict(X)
        predVal = gbm.predict(X_val)
        r2train =r2_score(train_set.label, predTrain)
        r2test =r2_score(valSet.label, predVal)
        r2ScoreDict[f][i] = {'train':r2train,'test':r2test}
        print('r2 scores', r2train,r2test)

        gbm.save_model(fileName)
        gbm.save_model('individualLGBMs/checkpoints/model_'+f+'_'+str(i)+'_'+str(round(r2test,3))+'.txt')
        del y, train_set, valSet#, predTrain, predVal


done preprocessing data
processing  ptend_t_0
r2 scores 0.771323975473696 0.8332045014521668
processing  ptend_t_1
r2 scores 0.8709142017506535 0.9075573778226572
processing  ptend_t_2
r2 scores 0.9427524347148162 0.9514701582676087
processing  ptend_t_3
r2 scores 0.9731769839151506 0.9729392962499134
processing  ptend_t_4
r2 scores 0.9785632846286109 0.9784633103590459
processing  ptend_t_5
r2 scores 0.9804873997771555 0.9810794468483718
processing  ptend_t_6
r2 scores 0.9807438627867612 0.9809312119657617
processing  ptend_t_7
r2 scores 0.9797383840579846 0.9797785853729524
processing  ptend_t_8
r2 scores 0.9770547008510726 0.9773046689698132
processing  ptend_t_9
r2 scores 0.9746517403116584 0.9749004087827285
processing  ptend_t_10
r2 scores 0.9721116425838738 0.9722975182898475
processing  ptend_t_11
r2 scores 0.9689676147421843 0.9690213391030311
processing  ptend_t_12
r2 scores 0.964191101918708 0.9645403825314751
processing  ptend_t_13
r2 scores 0.9566331050053378 0.95752525617

In [12]:
del X, locdata

In [11]:
for f in allT2:
    diff = r2ScoreDict[f][1]['test'] - r2ScoreDict[f][0]['test']
    if diff < 0:
        print(f,r2ScoreDict[f])

ptend_q0001_15 {0: {'train': 0.4938332254516111, 'test': -0.34495053829517364}, 1: {'train': 0.559133553543092, 'test': -1.0356995128321338}}
ptend_q0001_16 {0: {'train': 0.5441869731440663, 'test': 0.1608931137426396}, 1: {'train': 0.5944098206611488, 'test': 0.10711768595814786}}
ptend_q0002_28 {0: {'train': 0.9092368980867679, 'test': 0.8342647182257787}, 1: {'train': 0.8840045801874663, 'test': 0.833170042683148}}
ptend_q0003_15 {0: {'train': 0.4095786854165071, 'test': -3.646159547196275}, 1: {'train': 0.3781661194831788, 'test': -8.684712927088723}}
ptend_q0003_16 {0: {'train': 0.3622694417104786, 'test': -0.05991782844896143}, 1: {'train': 0.39846204090786486, 'test': -0.9424498870959703}}


In [10]:
import pickle

with open('individualLGBMs/r2ScoreDict.p', 'wb') as fp:
    pickle.dump(r2ScoreDict, fp, protocol=pickle.HIGHEST_PROTOCOL)

## more validation testing 

In [ ]:
for f in allT2:
    print('processing ',f)
    fileName = 'individualLGBMs/model_'+f+'.txt'
    gbm = lgb.Booster(model_file=fileName) if i != 0 else None
    predVal = gbm.predict(X_val)
    
    r2test =r2_score(y_val[f], predVal)
    r2ScoreDict[f][2] = {'test':r2test}
    print('r2 scores', r2test)

In [ ]:
del X_val, y_val

## submitting

In [4]:
testData = dd.read_parquet("test")
sampleSubmissions = dd.read_parquet('sampleSub')

In [5]:
testList = []
for i in range(testData.npartitions):
    testList.append(testData.get_partition(int(i)).compute())
test = pd.concat(testList)
X_test = test[allF]

sampSubList = []
for i in range(sampleSubmissions.npartitions):
    sampSubList.append(sampleSubmissions.get_partition(int(i)).compute())
sampleSub = pd.concat(sampSubList)

In [6]:
for f in allT2:
    print('processing ',f)
    fileName = 'individualLGBMs/model_'+f+'.txt'
    gbm = lgb.Booster(model_file=fileName)
    predVal = gbm.predict(X_test)
    
    sampleSub[f] = sampleSub[f] * predVal

processing  ptend_t_0
processing  ptend_t_1
processing  ptend_t_2
processing  ptend_t_3
processing  ptend_t_4
processing  ptend_t_5
processing  ptend_t_6
processing  ptend_t_7
processing  ptend_t_8
processing  ptend_t_9
processing  ptend_t_10
processing  ptend_t_11
processing  ptend_t_12
processing  ptend_t_13
processing  ptend_t_14
processing  ptend_t_15
processing  ptend_t_16
processing  ptend_t_17
processing  ptend_t_18
processing  ptend_t_19
processing  ptend_t_20
processing  ptend_t_21
processing  ptend_t_22
processing  ptend_t_23
processing  ptend_t_24
processing  ptend_t_25
processing  ptend_t_26
processing  ptend_t_27
processing  ptend_t_28
processing  ptend_t_29
processing  ptend_t_30
processing  ptend_t_31
processing  ptend_t_32
processing  ptend_t_33
processing  ptend_t_34
processing  ptend_t_35
processing  ptend_t_36
processing  ptend_t_37
processing  ptend_t_38
processing  ptend_t_39
processing  ptend_t_40
processing  ptend_t_41
processing  ptend_t_42
processing  ptend_t_4

In [7]:
sampleSub.to_parquet('sample_sub_LGBM_baseline2_shuff_allTrainF.parquet')

In [ ]:
del sampleSub, testData,sampleSubmissions, X_test, sampSubList, testList